# Object Detection for Airtime Scratchcard Digits

## Introduction
Hello everyone! In this project, I will focus on the task of converting Safaricom and Telkom scratchcard airtime images into digits that can be used to load airtime. In the past, there was an app called "RechargeKing" that allowed users to load airtime by capturing images of scratchcards. However, recent changes in the fonts of Safaricom and Telkom scratch cards have rendered the app obsolete as the developers did not update it.

## Problem Definition
The problem I aim to solve is the digit recognition from scratchcard images. I want to create a solution that can automatically extract the airtime digits from these images, allowing users to easily load airtime without manual input.

## Solution
To tackle this problem, I will leverage Nicholas Renotte's TensorFlow Object Detection course, which provides a comprehensive guide on using TensorFlow for object detection. This course will serve as my foundation to build an object detection model capable of recognizing and extracting the digits from scratchcard images. I will use Python for coding and follow the principles outlined in the course.

Let's get started!


# Steps to Achieve Object Detection

1. **Install and Setup**: We'll start by setting up our development environment and installing the necessary libraries, including TensorFlow.

2. **Collect Images and Label**: Next, we'll gather a dataset of images that contains the objects we want to detect. We'll also label these objects to train our model effectively.

3. **Training Models**: The heart of object detection is training models. I'll explain how to use pre-trained models or train custom ones to identify objects in images.

4. **Detecting Objects**: Once our models are trained, we'll apply them to real-world images to detect and locate objects accurately.

5. **Freezing and Conversion**: We'll discuss how to freeze and convert our trained model into a format suitable for deployment.

6. **Performance Tuning**: Object detection isn't just about accuracy; it's also about performance. We'll optimize our models for speed and efficiency.

7. **Packaging**: Finally, I'll guide you on packaging your object detection solution for practical use.



I plan on doing most of the work here then the heavy lifting will be done on collab


# 1. **Install and Setup**


In [ ]:
# # create venv environment 
# !python -m venv tfod

# # activate tfod env 
# !source tfod/bin/activate

In [ ]:
# # upgrade pip
# !python -m pip install --upgrade pip

# # install ipykernel
# !pip install ipykernel

# # assosiate ipykernel to tfod env where name is the name of your virtual environment
# !python -m ipykernel install --user --name=tfod

: 


# 2. **Collect Images and Label**


We will be using a number of scratchcard pictures(already used) and label them. right now i have about 50 so they should suffice


In [4]:
# install opencv
!pip install opencv-python

In [6]:
# import opencv 
import cv2

# import uuid 
import uuid

# import os 
import os

the IMAGES_PATH will host our images and corresponding labels

In [11]:
# make a folder for images
IMAGES_PATH =  os.path.join('tensorflow','workspace','images', 'collectedimages')

if not os.path.exists(IMAGES_PATH):
    if os.name == 'posix':
        !mkdir -p {IMAGES_PATH}
    if os.name == 'nt':
        !mkdir {IMAGES_PATH}


# 3. **Training Models**



# 4. **Detecting Objects**



# 5. **Freezing and Conversion**



# 6. **Performance Tuning**



# 7. **Packaging**